I have lost parquet with contextual data and have only standardized. This regenerates the original one.

In [1]:
import geopandas as gpd
import momepy as mm
import libpysal
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import mapclassify

from inequality.theil import Theil
from tqdm import tqdm
from momepy import limit_range
from sklearn import preprocessing

In [2]:
primary = pd.read_csv("/Users/martin/Dropbox/Academia/Data/Geo/Amsterdam/clustering/primary.csv", index_col=0)

In [4]:
geom = gpd.read_file("/Users/martin/Dropbox/Academia/Data/Geo/Amsterdam/clustering/geometry.gpkg", layer="tessellation")

/opt/miniconda3/envs/stable/lib/python3.8/site-packages/geopandas/geodataframe.py:422: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [7]:
%time queen3 = mm.sw_high(k=3, gdf=geom, ids='uID')

CPU times: user 4min 4s, sys: 15.1 s, total: 4min 19s
Wall time: 4min 25s


In [8]:
gdf = primary.set_index('uID')
spatial_weights = queen3
unique_id = 'uID'

In [9]:
means = {}
ranges = {}
theils = {}
simpsons = {}

for ch in gdf.columns:
    means[ch] = []
    ranges[ch] = []
    theils[ch] = []
    simpsons[ch] = []

In [10]:
gdf = gdf.fillna(0)  # normally does not happen, but to be sure
chars = gdf.columns

In [11]:
gdf['lcdMes'] = gdf.apply(
            lambda row: row.lcdMes if row.lcdMes >= 0 else 0,
            axis=1,
        )  # normally does not happen, but to be sure

In [12]:
def theil(y):
    y = np.array(y)
    n = len(y)
    plus = y + np.finfo('float').tiny * (y == 0)  # can't have 0 values
    yt = plus.sum(axis=0)
    s = plus / (yt * 1.0)
    lns = np.log(n * s)
    slns = s * lns
    t = sum(slns)
    return t

In [13]:
skewness = pd.DataFrame(index=chars)
for c in chars:
    skewness.loc[c, 'skewness'] = sp.stats.skew(gdf[c])
headtail = list(skewness.loc[skewness.skewness >= 1].index)
to_invert = skewness.loc[skewness.skewness <= -1].index

for inv in to_invert:
    gdf[inv + '_rev'] = gdf[inv].max() - gdf[inv]
inverted = [x for x in gdf.columns if '_rev' in x]
headtail = headtail + inverted
natural = [x for x in chars if x not in headtail]

In [14]:
def _simpson_di(data):

    def p(n, N):
        if n == 0:
            return 0
        return float(n) / N

    N = sum(data.values())

    return sum(p(n, N) ** 2 for n in data.values() if n != 0)

In [15]:
bins = {}
for c in headtail:
    bins[c] = mapclassify.HeadTailBreaks(gdf[c]).bins
for c in natural:
    bins[c] = mapclassify.gadf(gdf[c], method='NaturalBreaks')[1].bins

In [18]:
for index in tqdm(gdf.index, total=gdf.shape[0]):
    neighbours = [index]
    neighbours += spatial_weights.neighbors[index]
    
    vicinity = gdf.loc[neighbours]
    for ch in chars:
        values_list = vicinity[ch]
        idec = limit_range(values_list.tolist(), rng=(10, 90))
        iquar = limit_range(values_list.tolist(), rng=(25, 75))
        
        sample_bins = mapclassify.UserDefined(values_list, list(bins[ch]))
        counts = dict(zip(bins[ch], sample_bins.counts))
        simpsons[ch].append(_simpson_di(counts))
        
        means[ch].append(np.mean(iquar))
        ranges[ch].append(sp.stats.iqr(values_list, rng=(25, 75)))
        theils[ch].append(theil(idec))

  0%|          | 0/252385 [00:00<?, ?it/s]/opt/miniconda3/envs/stable/lib/python3.8/site-packages/mapclassify/classifiers.py:887: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
100%|██████████| 252385/252385 [5:23:14<00:00, 13.01it/s]   


In [24]:
contextual = pd.DataFrame(index=gdf.index)

In [26]:
for ch in chars:
    contextual[ch + '_meanIQ3'] = means[ch]
    contextual[ch + '_rangeIQ3'] = ranges[ch]
    contextual[ch + '_theilID3'] = theils[ch]
    contextual[ch + '_simpson'] = simpsons[ch]

In [27]:
contextual

,stcOri_meanIQ3,stcOri_rangeIQ3,stcOri_theilID3,sdcLAL_meanIQ3,sdcLAL_rangeIQ3,sdcLAL_theilID3,sdcAre_meanIQ3,sdcAre_rangeIQ3,sdcAre_theilID3,sscCCo_meanIQ3,...,sddAre_theilID3,sddAre_simpson,midRea_meanIQ3,midRea_rangeIQ3,midRea_theilID3,midRea_simpson,midAre_meanIQ3,midAre_rangeIQ3,midAre_theilID3,midAre_simpson
uID,,,,,,,,,,,,,,,,,,,,,
0,27.539986,0.029926,5.903783e-07,215.728286,1.064598,0.000012,28314.656812,446.137978,0.000124,0.774589,...,0.000000,1.000000,4.000000,0.00,0.000000,1.0,116032.091487,0.000000,0.000000e+00,1.000000
1,27.539986,0.029926,5.903783e-07,215.728286,1.064598,0.000012,28314.656812,446.137978,0.000124,0.774589,...,0.000000,1.000000,4.000000,0.00,0.000000,1.0,116032.091487,0.000000,0.000000e+00,1.000000
2,27.376524,15.287430,6.786512e-02,165.119895,60.349187,0.013950,10780.086406,7414.403564,0.054318,0.481731,...,0.036544,0.591837,44.000000,6.50,0.007313,1.0,458284.275492,10239.390277,2.104411e-04,1.000000
3,31.751999,14.433147,5.272121e-02,116.010516,90.511917,0.112148,5078.970289,8830.542588,0.392929,0.495462,...,0.000000,1.000000,44.000000,0.00,0.000000,1.0,458284.275492,0.000000,-1.110223e-16,1.000000
4,27.376524,15.287430,6.786512e-02,165.119895,60.349187,0.013950,10780.086406,7414.403564,0.054318,0.481731,...,0.036544,0.591837,44.000000,6.50,0.007313,1.0,458284.275492,10239.390277,2.104411e-04,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19452,37.493494,8.431445,6.527028e-03,80.571042,79.614308,0.097360,646.907689,539.572227,0.176220,0.216770,...,0.080639,1.000000,21.333333,10.00,0.028457,1.0,48777.894791,3620.826788,5.944435e-03,1.000000
19453,37.011002,8.552335,6.648164e-03,81.405301,81.483123,0.101711,811.882093,1471.286550,0.329740,0.276944,...,0.045213,1.000000,21.636364,10.00,0.045718,1.0,48375.580704,4774.807468,4.905983e-03,1.000000
19454,41.082484,2.350902,3.195354e-03,62.124290,84.110704,0.138473,611.970826,1352.082203,0.496895,0.274728,...,0.172782,0.957467,26.960000,26.25,0.110667,1.0,52006.586259,19655.190903,1.327803e-02,0.916824


In [28]:
contextual.to_parquet('/Users/martin/Dropbox/Academia/Data/Geo/Amsterdam/clustering/contextual.parquet')

In [22]:
chars.shape

(74,)

In [29]:
mm

<module 'momepy' from '/opt/miniconda3/envs/stable/lib/python3.8/site-packages/momepy/__init__.py'>